In [ ]:
from __future__ import division, print_function

import os
import sys
import time
import argparse
import shutil
import collections
import itertools
from glob import glob
import cPickle

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display
matplotlib.rc('savefig', dpi=100)

import numpy as np
import tensorflow as tf

In [ ]:
class Params(object):
        
    def __init__(self):
        """ Create a Params object.
        
        This is a convenience class for all parameters. We can either a) edit
        these parameters here or b) declare environment variables with the same
        names. Any existing environment variables take priority over the values
        listed here (this is useful for batch runs).
        
        Note: We define a sweep as a batch of sequences *that all have the same
        length.* Shorter sequences are wrapped in time to match the length of
        the longest sequence in a batch. From a modeling perspective, this means
        that short sequences contribute just as much as long sequences to
        overall loss.
        """
        
        self.parser = argparse.ArgumentParser()
        
        self._add_param('num_layers', 1)
        self._add_param('hidden_layer_size', 1024)
        self._add_param('init_scale', 0.1)
        self._add_param('dropout_keep_prob', 0.5)
        
        self._add_param('initial_learning_rate', 1.0)
        self._add_param('num_initial_sweeps', 300)
        self._add_param('num_sweeps_per_decay', 50)
        self._add_param('decay_factor', 0.5)
        self._add_param('max_global_grad_norm', 1.0)
        
        self._add_param('batch_size', 5)
        self._add_param('num_train_sweeps', 600)
        self._add_param('num_sweeps_per_summary', 7)
        self._add_param('num_sweeps_per_save', 7)
        
        self._add_param('data_dir', '~/Data/JIGSAWS/Suturing')
        self._add_param('data_filename', 'standardized_data.pkl')
        self._add_param('test_users', 'B')
        self._add_param('model_type', 'BidirectionalLSTM')
        self._add_param('gpu_ind', 0)
        self._add_param('gpu_mem_fraction', 1.0)
        
        self.args = self.parser.parse_args(args=[])
        
    def _add_param(self, name, default):
        """ Add a parameter.
        
        The parameter is added with the specified default, but this default
        is overridden by a corresponding environment variable if it exists.
        """
        
        param_type = type(default)
        self.parser.add_argument('--%s' % name, type=param_type, default=os.environ.get(name, default))

    @property
    def num_layers(self):
        """ An integer. The number of hidden layers. """
        return self.args.num_layers
    
    @property
    def hidden_layer_size(self):
        """ An integer. The number of hidden units per layer. """
        return self.args.hidden_layer_size
    
    @property
    def init_scale(self):
        """ A float. All model weights will be initialized using a uniform
        distribution over `[-init_scale, init_scale]`."""
        return self.args.init_scale
    
    @property
    def dropout_keep_prob(self):
        """ A float. The fraction of inputs to keep whenever dropout is
        applied. Dropout is applied to inputs/outputs of each time step, but
        never across time steps."""
        return self.args.dropout_keep_prob
    
    @property
    def initial_learning_rate(self):
        """ A float. """
        return self.args.initial_learning_rate
    
    @property
    def num_initial_sweeps(self):
        """ An integer. The number of initial sweeps during which the learning
        rate will remain fixed. """
        return self.args.num_initial_sweeps
    
    @property
    def num_sweeps_per_decay(self):
        """ An integer. The number of sweeps per decay once we exceed the
        number of initial sweeps. """
        return self.args.num_sweeps_per_decay
    
    @property
    def decay_factor(self):
        """ A float. The factor by which we'll decay after exceeding the number
        of initial sweeps. """
        return self.args.decay_factor
    
    @property
    def max_global_grad_norm(self):
        """ A float. All gradients are effectively concatenated to obtain the
        global norm. If this global norm exceeds `max_global_grad_norm`, then
        all gradients are scaled so that the global norm becomes
        `max_global_grad_norm`."""
        return self.args.max_global_grad_norm
    
    @property
    def batch_size(self):
        """ An integer. The number of sequences in a batch/sweep. """
        return self.args.batch_size
    
    @property
    def num_train_sweeps(self):
        """ An integer. """
        return self.args.num_train_sweeps
    
    @property
    def num_sweeps_per_summary(self):
        """ An integer. How often we'll compute summaries during training. """
        return self.args.num_sweeps_per_summary
    
    @property
    def num_sweeps_per_save(self):
        """ An integer. How often we'll save the model during training. """
        return self.args.num_sweeps_per_save
    
    @property
    def data_dir(self):
        """ A string. The directory containing our data. """
        return os.path.expanduser(self.args.data_dir)
    
    @property
    def data_filename(self):
        """ A string. The filename corresponding to the standardized Pickle
        file that contains the data. This file should reside in `data_dir`."""
        return self.args.data_filename
    
    @property
    def test_users(self):
        """ A list of strings. The users that make up the test set. """
        return self.args.test_users.split(' ')
    
    @property
    def model_type(self):
        """ A string. 'ForwardLSTM', 'ReverseLSTM', or 'BidirectionalLSTM'. """
        return self.args.model_type
    
    @property
    def gpu_ind(self):
        """ An integer or None. If None, use CPUs. Otherwise, `gpu_ind` indexes
        the `CUDA_VISIBLE_DEVICES` environment variable (which is already set if
        you have CUDA appropriately set up on your machine). For example, if
        `CUDA_VISIBLE_DEVICES` is '1,2', then `gpu_ind=1` will use GPU 2."""
        return self.args.gpu_ind
    
    @property
    def gpu_mem_fraction(self):
        """ A float. The fraction of memory for TensorFlow to acquire from the
        GPU specified by `gpu_ind`."""
        return self.args.gpu_mem_fraction
    
    @property
    def log_dir(self):
        """ A string. A log directory that's based on the parameters. For
        example, if the data directory is '~/Data/JIGSAWS', and we have only
        one test user 'B', then the log directory will look something like
        '~/Data/JIGSAWS/logs/BidirectionalLSTM_.../B'.
        
        Warning: This log directory will be deleted and replaced if it already
        exists, so it should be unique for parameters/users and should
        absolutely not be something generic like '~'.
        """
        
        params_str = '_'.join([self.model_type,
                               '%d' % self.num_layers,
                               '%03d' % self.hidden_layer_size,
                               '%02d' % self.batch_size,
                               '%.2f' % self.dropout_keep_prob,
                               '%.4f' % self.initial_learning_rate])
        
        test_users_str = '_'.join(self.test_users)
        
        return os.path.join(self.data_dir, 'logs', params_str, test_users_str)

In [ ]:
class Dataset(object):

    def __init__(self, pkl_path):
        """ Create a Dataset object from a standardized Pickle file.
        
        JIGSAWS and MISTIC contain similar underlying data, namely kinematics
        as input and surgical activity as output. This class loads a
        standardized Pickle file that can contain data for either dataset. See
        the properties below for a description of what the file must contain.
        
        Args:
            pkl_path: A string. A path to the standardized Pickle file.
        """

        with open(pkl_path, 'r') as f:
            self.pkl_dict = cPickle.load(f)

        assert all(seq.shape[1] - 1 == self.input_size for seq in self.all_data.values())
        
    def get_seqs_by_user(self, user):
        """ Get a list of sequences corresponding to a user.
        
        Args:
            user: A string.
        
        Returns:
            A list of sequences corresponding to `user`.
        """
        
        trial_names = sorted(self.user_to_trial_names[user])
        seqs = [self.all_data[trial_name] for trial_name in trial_names]
        return seqs
    
    def get_splits(self, test_users):
        """ Get all sequences, split into a training set and a testing set.
        
        Args:
            test_users: A list of strings.
        
        Returns:
            A tuple,
            train_seqs: A list of train sequences.
            test_seqs: A list of test sequences.
        """
        
        train_users = [user for user in self.all_users if user not in test_users]
        train_seqs = list(itertools.chain(*[self.get_seqs_by_user(user) for user in train_users]))
        test_seqs = list(itertools.chain(*[self.get_seqs_by_user(user) for user in test_users]))
        
        # Sanity check
        def seqs_are_same(seq1, seq2):
            return seq1.shape == seq2.shape and np.allclose(seq1, seq2, rtol=1e-3, atol=1e-3)
        for test_seq in test_seqs:
            assert any([seqs_are_same(test_seq, test_seq_) for test_seq_ in test_seqs])
            assert not any([seqs_are_same(test_seq, train_seq) for train_seq in train_seqs])
        
        return train_seqs, test_seqs
    
    @property
    def dataset_name(self):
        """ A string: the dataset name. """
        return self.pkl_dict['dataset_name']
    
    @property
    def classes(self):
        """ A list of strings: the class names. """
        return self.pkl_dict['classes']
    
    @property
    def num_classes(self):
        """ An integer: the number of classes. """
        return self.pkl_dict['num_classes']
    
    @property
    def all_users(self):
        """ A list of strings, each representing a user. """
        return self.pkl_dict['all_users']
    
    @property
    def all_trial_names(self):
        """ A list of strings: all trial names over all users. """
        return self.pkl_dict['all_trial_names']
    
    @property
    def user_to_trial_names(self):
        """ A dictionary mapping users to trial-name lists. """
        return self.pkl_dict['user_to_trial_names']
    
    @property
    def all_data(self):
        """ A dictionary mapping trial names to NumPy arrays. Each NumPy
            array has shape `[duration, input_size+1]`, with the last
            column being class labels. """
        return self.pkl_dict['all_data']
    
    @property
    def col_names(self):
        """ A list of strings: the column names for each data column. """
        return self.pkl_dict['col_names']
    
    @property
    def input_size(self):
        """ An integer: the number of inputs per time step. """
        return self.all_data.values()[0].shape[1] - 1

In [ ]:
def normalize_seq(seq):
    """ Normalize a sequence by centering/scaling columns.

    Args:
        seq: A 2-D NumPy array with shape `[duration, size]`.
    
    Returns:
        A 2-D NumPy array with the same shape, but with all columns
        having mean 0 and standard deviation 1.
    """
    
    mu = seq.mean(axis=0, keepdims=True)
    sigma = seq.std(axis=0, keepdims=True)
    normalized_seq = (seq - mu) / sigma
    return normalized_seq

In [ ]:
def prepare_raw_seq(seq):
    """ Prepare a raw sequence for training/testing.
    
    This function a) splits a raw sequence into input and label sequences; b)
    prepares a reset sequence (for handling RNN state resets); and c)
    normalizes each input sequence.
    
    Args:
        seq: A 2-D NumPy array with shape `[duration, num_inputs + 1]`.
            The last column stores labels.
            
    Returns:
        A tuple,
        input_seq: A 2-D float32 NumPy array with shape
            `[duration, num_inputs]`. A normalized input sequence.
        reset_seq: A 2-D bool NumPy array with shape `[duration, 1]`.
        label_seq: A 2-D int32 NumPy array with shape `[duration, 1]`.
    """
    
    input_seq = seq[:, :-1].astype(np.float)
    input_seq = normalize_seq(input_seq).astype(np.float32)
    duration = input_seq.shape[0]
    reset_seq = np.eye(1, duration, dtype=np.bool).T
    label_seq = seq[:, -1:].astype(np.int32)
    return input_seq, reset_seq, label_seq

In [ ]:
def one_hot(labels, num_classes):
    """ Convert labels to one-hot encodings.
    
    Args:
        labels: A NumPy array of nonnegative labels.
    
    Returns:
        A NumPy array with shape `labels.shape + [num_classes]`. That is,
        the same shape is retained, but one axis is added for the one-hot
        encodings.
    """
    
    encoding_matrix = np.zeros([labels.size, num_classes])
    encoding_matrix[range(labels.size), labels.flatten()] = 1
    encodings = encoding_matrix.reshape(list(labels.shape) + [num_classes])
    return encodings

In [ ]:
def seq_ind_generator(num_seqs, shuffle=True):
    """ A sequence-index generator.
    
    Args:
        num_seqs: An integer. The number of sequences we'll be indexing.
        shuffle: A boolean. If true, randomly shuffle indices epoch by epoch.
    
    Yields:
        An integer in `[0, num_seqs)`.    
    """
    
    seq_inds = range(num_seqs)
    while True:
        if shuffle:
            np.random.shuffle(seq_inds)
        for seq_ind in seq_inds:
            yield seq_ind

In [ ]:
def sweep_generator(seq_list_list, batch_size, shuffle=False, num_sweeps=None):
    """ Generate sweeps.
    
    Let's define a sweep as a collection of `batch_size` sequences that
    continue together through time until all sequences in the batch have been
    exhausted. Short sequences grow by being wrapped in time.
    
    Simplified example: pretend sequences are 1-D arrays, and that we have
    `seq_list = [[1, 0], [1, 0, 0]]`. Then
    `sweep_generator([seq_list], 3, shuffle=False)` would yield
    `[ [[1, 0, 1], [1, 0, 0], [1, 0, 1]] ]`.
    
    Args:
        seq_list_list: A list of sequence lists. The sequences in
            `seq_list_list[0]` should correspond to the sequences in
            `seq_list_list[1]`, in `seq_list_list[2]`, etc. Their durations
            should be the same, but data types can differ. All sequences
            should be 2-D and have time running along axis 0.
        batch_size: An integer. The number of sequences in a batch.
        shuffle: A boolean. If true, shuffle sequences epoch by epoch as we
            populate sweeps.
        num_sweeps: An integer. The number of sweeps to visit before the
            generator is exhaused. If None, generate sweeps forever.
    
    Yields:
        A list with the same length as `seq_list_list`. This contains a sweep
        for the 1st seq list, a sweep for the 2nd seq list, etc., each sweep
        being a NumPy array with shape `[batch_size, duration, ?]`.
    """
    
    if num_sweeps is None:
        num_sweeps = np.inf
    
    seq_durations = [len(seq) for seq in seq_list_list[0]]
    num_seqs = len(seq_list_list[0])
    seq_ind_gen = seq_ind_generator(num_seqs, shuffle=shuffle)
    
    for seq_list in seq_list_list:
        assert len(seq_list) == num_seqs
        assert [len(seq) for seq in seq_list] == seq_durations
    
    num_sweeps_visited = 0
    while num_sweeps_visited < num_sweeps:
        
        new_seq_ind = [seq_ind_gen.next() for _ in xrange(batch_size)]
        new_seq_durations = [seq_durations[i] for i in new_seq_ind]
        longest_duration = np.max(new_seq_durations)
        pad = lambda seq: np.pad(seq, [[0, longest_duration-len(seq)], [0, 0]], mode='wrap')
        
        new_sweep_list = []
        for seq_list in seq_list_list:
            new_seq_list = [seq_list[i] for i in new_seq_ind]
            new_sweep = np.asarray([pad(seq) for seq in new_seq_list])
            new_sweep_list.append(new_sweep)
        
        yield new_sweep_list
        num_sweeps_visited += 1

In [ ]:
class LSTM(object):
    
    def __init__(self, inputs, resets, training, params):
        """ Create a long short-term memory RNN.
        
        This maps RNN inputs to RNN outputs. Computing predictions from the RNN
        outputs needs to happen elsewhere.
        
        Args:
            inputs: A 3-D float32 Tensor with shape
                `[batch_size, duration, input_size]`.
            resets: A 3-D bool Tensor with shape
                `[batch_size, duration, 1]`. These indicate when sequences
                reset (to handle states appropriately with sequences that have
                been wrapped to form a sweep).
            training: A 0-D bool Tensor. When False, dropout won't be applied.
            params: A Params object.
        """
        
        self.inputs = inputs
        self.resets = resets
        self.training = training
        self.input_size = inputs.get_shape().as_list()[2]
        
        self.num_layers = params.num_layers
        self.hidden_layer_size = params.hidden_layer_size
        self.init_scale = params.init_scale
        self.dropout_keep_prob = params.dropout_keep_prob
        
        batch_states_shape = tf.pack([tf.shape(inputs)[0], 2*self.hidden_layer_size])
        self._batch_start_states = tf.zeros(batch_states_shape, dtype=tf.float32)
        keep_prob = tf.cond(training, lambda: tf.constant(self.dropout_keep_prob), lambda: tf.constant(1.0))
        
        initializer = tf.random_uniform_initializer(-self.init_scale, self.init_scale)
        with tf.variable_scope('LSTM', initializer=initializer):
            
            # We need to swap the batch, time axes since tf.scan will split
            # along dimension 0.
            inputs = tf.transpose(inputs, [1, 0, 2])
            resets = tf.transpose(resets, [1, 0, 2])
            
            states_list = []
            prev_layer_outputs = tf.nn.dropout(inputs, keep_prob)
            for layer in xrange(self.num_layers):
                
                def fixed_size_lstm_block(c_prev_and_m_prev, x_and_r):
                    block_input_size = self.input_size if layer == 0 else self.hidden_layer_size
                    return self._lstm_block(c_prev_and_m_prev, x_and_r, block_input_size)
                
                x_and_r = tf.concat(2, [prev_layer_outputs, tf.cast(resets, tf.float32)])
                with tf.variable_scope('layer%d' % layer):
                    c_and_m = tf.scan(fixed_size_lstm_block, x_and_r, initializer=self._batch_start_states)
                states_list.append(c_and_m)
                prev_layer_outputs = tf.nn.dropout(c_and_m[:, :, self.hidden_layer_size:], keep_prob)
                
            _states = tf.concat(2, [tf.expand_dims(states, 2) for states in states_list])
            
            # Now put the batch, time axes back.
            self._states = tf.transpose(_states, [1, 0, 2, 3])
            self._outputs = tf.transpose(prev_layer_outputs, [1, 0, 2])
        
    def _lstm_block(self, c_prev_and_m_prev, x_and_r, block_input_size):
        """ LSTM block.
        
        This implementation uses a forget gate and peephole connections. Also,
        sequence resets `r` are used here to handle state resets internally
        instead of externally.

        Args:
            c_prev_and_m_prev: A 2-D float32 Tensor with shape
                `[batch_size, 2*hidden_layer_size]`.
            x_and_r: A 2-D float32 Tensor with shape
                `[batch_size, block_input_size+1]`. It's a concatenation of
                the block's inputs with sequence-reset booleans (though with
                type float32). In the case of multiple layers, the inputs are
                the previous layer's outputs.
            block_input_size: An integer.

        Returns:
            The updated state c_and_m, with the same shape as
            `c_prev_and_m_prev`.
        """
        
        def xmul(tensor, weights_name):
            W = tf.get_variable(weights_name, shape=[block_input_size, self.hidden_layer_size])
            return tf.matmul(tensor, W)
        
        def mmul(tensor, weights_name):
            W = tf.get_variable(weights_name, shape=[self.hidden_layer_size, self.hidden_layer_size])
            return tf.matmul(tensor, W)
        
        def diagcmul(tensor, weights_name):
            w = tf.get_variable(weights_name, shape=[self.hidden_layer_size])
            return tensor*w

        def bias(name):
            b = tf.get_variable(name, shape=[self.hidden_layer_size], initializer=tf.constant_initializer(0.0))
            return b
        
        x = x_and_r[:, :block_input_size]
        r = tf.cast(x_and_r[:, block_input_size], tf.bool)
        
        # If r[i] is True, revert back to initial states. Otherwise, keep
        # the states from the previous time step.
        c_prev_and_m_prev = tf.select(r, self._batch_start_states, c_prev_and_m_prev)
        c_prev, m_prev = tf.split(1, 2, c_prev_and_m_prev)
        
        x_tilde = tf.tanh( xmul(x, 'W_xx') + mmul(m_prev, 'W_xm') + bias('b_x') )
        i = tf.sigmoid( xmul(x, 'W_ix') + mmul(m_prev, 'W_im') + diagcmul(c_prev, 'w_ic') + bias('b_i') )
        f = tf.sigmoid( xmul(x, 'W_fx') + mmul(m_prev, 'W_fm') + diagcmul(c_prev, 'w_fc') + bias('b_f') )
        c = x_tilde*i + c_prev*f
        o = tf.sigmoid( xmul(x, 'W_ox') + mmul(m_prev, 'W_om') + diagcmul(c, 'w_oc') + bias('b_o') )
        m = o*tf.tanh(c)
        
        c_and_m = tf.concat(1, [c, m])
        return c_and_m
    
    @property
    def states(self):
        """ A 4-D float32 Tensor with shape
        `[batch_size, duration, num_layers, hidden_layer_size]`. """
        return self._states
    
    @property
    def outputs(self):
        """ A 3-D float32 Tensor with shape
        `[batch_size, duration, hidden_layer_size]`. """
        return self._outputs

In [ ]:
class LSTMModel(object):
    
    def __init__(self, input_size, target_size, params):
        """ A base class for LSTM models that includes predictions and loss.
        
        Args:
            input_size: An integer. The number of inputs per time step.
            target_size: An integer. The dimensionality of the one-hot
                encoded targets.
            params: A Params object.       
        """
        
        self.input_size = input_size
        self.target_size = target_size
        self.init_scale = params.init_scale
        self.params = params
        
        self._inputs = tf.placeholder(tf.float32, shape=[None, None, input_size], name='inputs')
        self._resets = tf.placeholder(tf.bool, shape=[None, None, 1], name='resets')
        self._targets = tf.placeholder(tf.float32, shape=[None, None, target_size], name='targets')
        self._training = tf.placeholder(tf.bool, shape=[], name='training')
        
        outputs = self._compute_rnn_outputs()
        output_size = self._compute_rnn_output_size()
        
        initializer = tf.random_uniform_initializer(-self.init_scale, self.init_scale)
        with tf.variable_scope('logits', initializer=initializer):
            W = tf.get_variable('W', shape=[output_size, self.target_size])
            b = tf.get_variable('b', shape=[self.target_size])
            outputs_matrix = tf.reshape(outputs, [-1, output_size])
            logits = tf.nn.xw_plus_b(outputs_matrix, W, b)
            batch_size, duration, _ = tf.unpack(tf.shape(self.inputs))
            logits_shape = tf.pack([batch_size, duration, self.target_size])
            self._logits = tf.reshape(logits, logits_shape, name='logits')
        
        with tf.variable_scope('loss'):
            logits = tf.reshape(self.logits, [-1, self.target_size])
            targets = tf.reshape(self.targets, [-1, self.target_size])
            cross_entropies = tf.nn.softmax_cross_entropy_with_logits(logits, targets)
            self._loss = tf.reduce_mean(cross_entropies, name='loss')
            
    def _compute_rnn_outputs(self):
        """ Compute RNN outputs.
        
        Returns:
            A 3-D float32 Tensor with shape `[batch_size, duration, output_size]`.
        """
        
        raise NotImplementedError()
    
    def _compute_rnn_output_size(self):
        """ Compute RNN output size.
        
        It's not always the same as `hidden_layer_size`: in the
        BidirectionalLSTM case, it's `2*hidden_layer_size`.
        
        Returns:
            An integer.
        """
        
        raise NotImplementedError()

    @property
    def inputs(self):
        """ A 3-D float32 Placeholder with shape `[batch_size, duration, input_size]`. """
        return self._inputs
    
    @property
    def resets(self):
        """ A 3-D bool Placeholder with shape `[batch_size, duration, 1]`. """
        return self._resets
    
    @property
    def targets(self):
        """ A 3-D float32 Placeholder with shape `[batch_size, duration, target_size]`. """
        return self._targets
    
    @property
    def training(self):
        """ A 0-D bool Placeholder. """
        return self._training
    
    @property
    def logits(self):
        """ A 3-D float32 Tensor with shape `[batch_size, duration, target_size]`. """
        return self._logits
    
    @property
    def loss(self):
        """ A 0-D float32 Tensor. """
        return self._loss

In [ ]:
class ForwardLSTMModel(LSTMModel):
    
    def __init__(self, *args):
        """ Create a forward LSTM model.
        
        Args:
            See `LSTMModel`.
        """
        super(ForwardLSTMModel, self).__init__(*args)
    
    def _compute_rnn_outputs(self):
        self._fw_lstm = LSTM(self.inputs, self.resets, self.training, self.params)
        return self._fw_lstm.outputs
    
    def _compute_rnn_output_size(self):
        return self._fw_lstm.hidden_layer_size

In [ ]:
class ReverseLSTMModel(LSTMModel):
    
    def __init__(self, *args):
        """ Create a reverse LSTM model.
        
        Args:
            See `LSTMModel`.
        """
        super(ReverseLSTMModel, self).__init__(*args)
    
    def _compute_rnn_outputs(self):
        reversed_inputs = tf.reverse(self.inputs, [False, True, False])
        reversed_resets = tf.reverse(self.resets, [False, True, False])
        self._rv_lstm = LSTM(reversed_inputs, reversed_resets, self.training, self.params)
        outputs = tf.reverse(self._rv_lstm.outputs, [False, True, False])
        return outputs
    
    def _compute_rnn_output_size(self):
        return self._rv_lstm.hidden_layer_size

In [ ]:
class BidirectionalLSTMModel(LSTMModel):
    
    def __init__(self, *args):
        """ Create a bidirectional LSTM model.
        
        Args:
            See `LSTMModel`.
        """
        super(BidirectionalLSTMModel, self).__init__(*args)
    
    def _compute_rnn_outputs(self):
        
        reversed_inputs = tf.reverse(self.inputs, [False, True, False])
        reversed_resets = tf.reverse(self.resets, [False, True, False])
        with tf.variable_scope('fw'):
            self._fw_lstm = LSTM(self.inputs, self.resets, self.training, self.params)
        with tf.variable_scope('rv'):
            self._rv_lstm = LSTM(reversed_inputs, reversed_resets, self.training, self.params)
        
        fw_outputs = self._fw_lstm.outputs
        rv_outputs = tf.reverse(self._rv_lstm.outputs, [False, True, False])
        outputs = tf.concat(2, [fw_outputs, rv_outputs])
        return outputs
    
    def _compute_rnn_output_size(self):
        return self._fw_lstm.hidden_layer_size + self._rv_lstm.hidden_layer_size

In [ ]:
def piecewise_constant(x, boundaries, values):
    """ Piecewise constant function.
    
    Arguments:
        x: A 0-D Tensor.
        boundaries: A 1-D NumPy array with strictly increasing entries.
        values: A 1-D NumPy array that specifies the values for the intervals
            defined by `boundaries`. (It should therefore have one more entry
            than `boundaries`.)
    
    Returns: A 0-D Tensor. Its value is `values[0]` when `x <= boundaries[0]`,
        `values[1]` when `x > boundaries[0]` and `x <= boundaries[1]`, ..., and
        values[-1] when `x > boundaries[-1]`.
    """
    
    pred_fn_pairs = {}
    pred_fn_pairs[x <= boundaries[0]] = lambda: tf.constant(values[0])
    pred_fn_pairs[x > boundaries[-1]] = lambda: tf.constant(values[-1])
    for lower, upper, value in zip(boundaries[:-1], boundaries[1:], values[1:-1]):
        # We need to bind value here; can do this with lambda value=value: ...
        pred_fn_pairs[(x > lower) & (x <= upper)] = lambda value=value: tf.constant(value)
    
    return tf.case(pred_fn_pairs, lambda: tf.constant(values[0]), exclusive=True)

In [ ]:
class Optimizer(object):
    
    def __init__(self, loss, params):
        """ Create an optimizer.
        
        If the global norm over all gradients is greater than
        `max_global_grad_norm`, then scale all gradients so that the global
        norm becomes `max_global_grad_norm`. Then update using vanilla SGD with
        a staircase learning-rate schedule which a) maintains the initial
        learning rate for `params.num_initial_sweeps` sweeps and then b) decays
        the learning rate by a factor of `params.decay_factor` every
        `params.num_sweeps_per_decay` sweeps.
        
        Args:
            loss: A 0-D float32 Tensor.
            params: A Params object.
        """
        
        self.loss = loss
        self.initial_learning_rate = params.initial_learning_rate
        self.num_initial_sweeps = params.num_initial_sweeps
        self.num_sweeps_per_decay = params.num_sweeps_per_decay
        self.decay_factor = params.decay_factor
        self.max_global_grad_norm = params.max_global_grad_norm
        self.num_train_sweeps = params.num_train_sweeps
        
        self._trainables = tf.trainable_variables()
        self._raw_grads = tf.gradients(loss, self.trainables)
        self._scaled_grads, _ = tf.clip_by_global_norm(self.raw_grads,
                                                       clip_norm=self.max_global_grad_norm)
        
        self._num_sweeps_visited = tf.Variable(0, trainable=False, name='num_sweeps_visited', dtype=tf.int32)
        boundaries = np.arange(self.num_initial_sweeps, self.num_train_sweeps,
                               self.num_sweeps_per_decay, dtype=np.int32)
        values = [self.initial_learning_rate * self.decay_factor**i
                  for i in xrange(len(boundaries)+1)]
        self._learning_rate = piecewise_constant(self.num_sweeps_visited, boundaries, values)
        
        optimizer = tf.train.GradientDescentOptimizer(self.learning_rate)
        grad_var_pairs = zip(self.scaled_grads, self.trainables)
        self._optimize_op = optimizer.apply_gradients(grad_var_pairs, global_step=self.num_sweeps_visited)
        
    @property
    def num_sweeps_visited(self):
        """ A 0-D int32 Tensor. """
        return self._num_sweeps_visited
    
    @property
    def learning_rate(self):
        """ A 0-D float32 Tensor. """
        return self._learning_rate
    
    @property
    def trainables(self):
        """ A list of trainable variables that will be updated. """
        return self._trainables
    
    @property
    def raw_grads(self):
        """ A list of raw gradient Tensors. """
        return self._raw_grads
    
    @property
    def scaled_grads(self):
        """ A list of scaled gradient Tensors. """
        return self._scaled_grads
    
    @property
    def optimize_op(self):
        """ An Operation that takes one optimization step. """
        return self._optimize_op

In [ ]:
def predict(sess, model, input_seqs, reset_seqs):
    """ Compute prediction sequences from input sequences.
    
    Args:
        sess: A Session.
        model: An LSTMModel.
        input_seqs: A list of input sequences, each a float32 NumPy array with
            shape `[duration, input_size]`.
        reset_seqs: A list of reset sequences, each a bool NumPy array with
            shape `[duration, 1]`.
    
    Returns:
        A list of prediction sequences, each a NumPy array with shape
        `[duration, 1]`, containing predicted labels for each time step.
    """
    
    batch_size = len(input_seqs)
    seq_durations = [len(seq) for seq in input_seqs]
    input_sweep, reset_sweep = sweep_generator([input_seqs, reset_seqs], batch_size=batch_size).next()
    
    logit_sweep = sess.run(model.logits, feed_dict={model.inputs: input_sweep,
                                                    model.resets: reset_sweep,
                                                    model.training: False})
    
    logit_seqs = [seq[:duration] for (seq, duration) in zip(logit_sweep, seq_durations)]
    prediction_seqs = [np.argmax(seq, axis=1).reshape(-1, 1) for seq in logit_seqs]
    
    return prediction_seqs

In [ ]:
def compute_accuracy(prediction_seq, label_seq):
    """ Compute accuracy, the fraction of correct predictions.
    
    Args:
        prediction_seq: A 2-D int NumPy array with shape
            `[duration, 1]`.
        label_seq: A 2-D int NumPy array with the same shape.
    
    Returns:
        A float.
    """
    
    return np.mean(prediction_seq == label_seq, dtype=np.float)

In [ ]:
def compute_edit_distance(prediction_seq, label_seq):
    """ Compute segment-level edit distance.
    
    First, transform each sequence to the segment level by replacing any
    repeated, adjacent labels with one label. Second, compute the edit distance
    (Levenshtein distance) between the two segment-level sequences.
    
    Simplified example: pretend each input sequence is only 1-D, with
    `prediction_seq = [1, 3, 2, 2, 3]` and `label_seq = [1, 2, 2, 2, 3]`.
    The segment-level equivalents are `[1, 3, 2, 3]` and `[1, 2, 3]`, resulting
    in an edit distance of 1.
    
    Args:
        prediction_seq: A 2-D int NumPy array with shape
            `[duration, 1]`.
        label_seq: A 2-D int NumPy array with shape
            `[duration, 1]`.
    
    Returns:
        A nonnegative integer, the number of operations () to transform the
        segment-level version of `prediction_seq` into the segment-level
        version of `label_seq`.
    """
    
    def edit_distance(seq1, seq2):

        seq1 = [-1] + list(seq1)
        seq2 = [-1] + list(seq2)

        dist_matrix = np.zeros([len(seq1), len(seq2)], dtype=np.int)
        dist_matrix[:, 0] = np.arange(len(seq1))
        dist_matrix[0, :] = np.arange(len(seq2))

        for i in xrange(1, len(seq1)):
            for j in xrange(1, len(seq2)):
                if seq1[i] == seq2[j]:
                    dist_matrix[i, j] = dist_matrix[i-1, j-1]
                else:
                    operation_dists = [dist_matrix[i-1, j], dist_matrix[i, j-1], dist_matrix[i-1, j-1]]
                    dist_matrix[i, j] = np.min(operation_dists) + 1

        return dist_matrix[-1, -1]
    
    def segment_level(seq):
        segment_level_seq = []
        for label in seq.flatten():
            if len(segment_level_seq) == 0 or segment_level_seq[-1] != label:
                segment_level_seq.append(label)
        return segment_level_seq
    
    return edit_distance(segment_level(prediction_seq), segment_level(label_seq))

In [ ]:
def compute_metrics(prediction_seqs, label_seqs):
    """ Compute metrics averaged over sequences.
    
    Args:
        prediction_seqs: A list of int NumPy arrays, each with shape
            `[duration, 1]`.
        label_seqs: A list of int NumPy arrays, each with shape
            `[duration, 1]`.
    
    Returns:
        A tuple,
        mean_accuracy: A float, the average over all sequences of the
            accuracies computed on a per-sequence basis.
        mean_edit_distance: A float, the average over all sequences of
            edit distances computed on a per-sequence basis.
    """
    
    accuracies = [compute_accuracy(pred_seq, label_seq)
                  for (pred_seq, label_seq) in zip(prediction_seqs, label_seqs)]
    edit_dists = [compute_edit_distance(pred_seq, label_seq)
                  for (pred_seq, label_seq) in zip(prediction_seqs, label_seqs)]
    return np.mean(accuracies, dtype=np.float), np.mean(edit_dists, dtype=np.float)

In [ ]:
def plot_seq(label_seq, num_classes, y_value):
    """ Plot a label sequence.
    
    The sequence will be shown using a horizontal colored line, with colors
    corresponding to classes.
    
    Args:
        label_seq: An int NumPy array with shape `[duration, 1]`.
        num_classes: An integer.
        y_value: A float. The y value at which the horizontal line will sit.
    """
    
    label_seq = label_seq.flatten()
    x = np.arange(0, label_seq.size)
    y = y_value*np.ones(label_seq.size)
    plt.scatter(x, y, c=label_seq, marker='|', lw=2, vmin=0, vmax=num_classes)

In [ ]:
def visualize(prediction_seqs, label_seqs, num_classes, fig_width=6.5, fig_height_per_seq=0.5):
    """ Visualize predictions vs. ground truth.
    
    Args:
        prediction_seqs: A list of int NumPy arrays, each with shape
            `[duration, 1]`.
        label_seqs: A list of int NumPy arrays, each with shape `[duration, 1]`.
        num_classes: An integer.
        fig_width: A float. Figure width (inches).
        fig_height_per_seq: A float. Figure height per sequence (inches).
    
    Returns:
        A tuple of the created figure, axes.
    """
    
    num_seqs = len(label_seqs)
    max_seq_length = max([seq.shape[0] for seq in label_seqs])
    figsize = (fig_width, num_seqs*fig_height_per_seq)
    fig, axes = plt.subplots(nrows=num_seqs, ncols=1, sharex=True, figsize=figsize)
    
    for pred_seq, label_seq, ax in zip(prediction_seqs, label_seqs, axes):
        plt.sca(ax)
        plot_seq(label_seq, num_classes, 1)
        plot_seq(pred_seq, num_classes, -1)
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        plt.xlim(0, max_seq_length)
        plt.ylim(-2.75, 2.75)
        plt.tight_layout()
        
    return fig, axes

In [ ]:
def train(sess, model, optimizer, train_input_seqs, train_reset_seqs, train_label_seqs,
          test_input_seqs, test_reset_seqs, test_label_seqs, params):
    """ Train a model.
    
    This function trains a model, and during training exports TensorBoard
    summaries for both the train set and the test set.
    
    Args:
        sess: A Session.
        model: An LSTMModel.
        optimizer: An Optimizer.
        train_input_seqs: A list of 2-D float NumPy arrays, each with shape
            `[duration, input_size]`.
        train_reset_seqs: A list of 2-D bool NumPy arrays, each with shape
            `[duration, 1]`.
        train_label_seqs: A list of 2-D int NumPy arrays, each with shape
            `[duration, 1]`.
        test_input_seqs: A list of 2-D float NumPy arrays, each with shape
            `[duration, input_size]`.
        test_reset_seqs: A list of 2-D bool NumPy arrays, each with shape
            `[duration, 1]`.
        test_label_seqs: A list of 2-D int NumPy arrays, each with shape
            `[duration, 1]`.
        params: A Params object.
    """
    
    log_dir = params.log_dir
    batch_size = params.batch_size
    num_train_sweeps = params.num_train_sweeps
    num_sweeps_per_summary = params.num_sweeps_per_summary
    num_sweeps_per_save = params.num_sweeps_per_save
    
    tf.scalar_summary('learning_rate', optimizer.learning_rate)

    ema = tf.train.ExponentialMovingAverage(decay=0.5)
    update_train_loss_ema = ema.apply([model.loss])
    train_loss_ema = ema.average(model.loss)
    tf.scalar_summary('train_loss_ema', train_loss_ema)
    
    train_accuracy = tf.placeholder(tf.float32, name='train_accuracy')
    train_edit_dist = tf.placeholder(tf.float32, name='train_edit_dist')
    test_accuracy = tf.placeholder(tf.float32, name='test_accuracy')
    test_edit_dist = tf.placeholder(tf.float32, name='test_edit_dist')
    values = [train_accuracy, train_edit_dist, test_accuracy, test_edit_dist]
    tags = [value.op.name for value in values]
    tf.scalar_summary(tags, tf.pack(values))
    
    summary_op = tf.merge_all_summaries()
    
    if os.path.exists(log_dir):
        shutil.rmtree(log_dir)
    summary_writer = tf.train.SummaryWriter(logdir=log_dir, graph=sess.graph)
    saver = tf.train.Saver()
    
    sess.run(tf.initialize_all_variables())
    
    num_sweeps_visited = 0
    start_time = time.time()
    train_gen = sweep_generator([train_input_seqs, train_reset_seqs, train_label_seqs],
                                batch_size=batch_size, shuffle=True, num_sweeps=None)
    while num_sweeps_visited <= num_train_sweeps:
        
        if num_sweeps_visited % num_sweeps_per_summary == 0:
            
            train_prediction_seqs = predict(sess, model, train_input_seqs, train_reset_seqs)
            train_accuracy_, train_edit_dist_ = compute_metrics(train_prediction_seqs, train_label_seqs)
            test_prediction_seqs = predict(sess, model, test_input_seqs, test_reset_seqs)
            test_accuracy_, test_edit_dist_ = compute_metrics(test_prediction_seqs, test_label_seqs)
            summary = sess.run(summary_op, feed_dict={train_accuracy: train_accuracy_,
                                                      train_edit_dist: train_edit_dist_,
                                                      test_accuracy: test_accuracy_,
                                                      test_edit_dist: test_edit_dist_})
            summary_writer.add_summary(summary, global_step=num_sweeps_visited)
            
            with open(os.path.join(log_dir, 'status.txt'), 'w') as f:
                line = '%05.1f      ' % ((time.time() - start_time)/60)
                line += '%04d      ' % num_sweeps_visited
                line += '%.6f  %08.3f     ' % (train_accuracy_, train_edit_dist_)
                line += '%.6f  %08.3f     ' % (test_accuracy_, test_edit_dist_)
                print(line, file=f)
            
            with open(os.path.join(log_dir, 'test_label_seqs.pkl'), 'w') as f:
                cPickle.dump(test_label_seqs, f)
            with open(os.path.join(log_dir, 'test_prediction_seqs.pkl'), 'w') as f:
                cPickle.dump(test_prediction_seqs, f)
                
            fig, axes = visualize(test_prediction_seqs, test_label_seqs, model.target_size)
            axes[0].set_title(line)
            display.display(fig)
            display.clear_output(wait=True)
                
        if num_sweeps_visited % num_sweeps_per_save == 0:
            saver.save(sess, os.path.join(log_dir, 'model.ckpt'))
        
        train_inputs, train_resets, train_labels = train_gen.next()
        # We squeeze here because otherwise the targets would have shape
        # [batch_size, duration, 1, num_classes].
        train_targets = one_hot(train_labels, model.target_size).squeeze(axis=2)
        _, _, num_sweeps_visited = sess.run(
            [optimizer.optimize_op, update_train_loss_ema, optimizer.num_sweeps_visited],
            feed_dict={model.inputs: train_inputs,
                       model.resets: train_resets,
                       model.targets: train_targets,
                       model.training: True})

In [ ]:
def run_training(params):
    """ Run training for a single configuration.
    
    Args:
        params: A Params object.
    """
    
    data_dir = params.data_dir
    data_filename = params.data_filename
    test_users = params.test_users
    model_type = params.model_type
    gpu_ind = params.gpu_ind
    gpu_mem_fraction = params.gpu_mem_fraction
    log_dir = params.log_dir
    
    pkl_path = os.path.join(data_dir, data_filename)
    if not os.path.exists(data_dir):
        raise ValueError('data_dir does not exist.')
    if not os.path.exists(pkl_path):
        raise ValueError('data_filename does not reside in data_dir.')
    
    dataset = Dataset(pkl_path)
    train_raw_seqs, test_raw_seqs = dataset.get_splits(test_users)
    train_triplets = [prepare_raw_seq(seq) for seq in train_raw_seqs]
    train_input_seqs, train_reset_seqs, train_label_seqs = zip(*train_triplets)
    test_triplets = [prepare_raw_seq(seq) for seq in test_raw_seqs]
    test_input_seqs, test_reset_seqs, test_label_seqs = zip(*test_triplets)
        
    Model = eval(model_type + 'Model')
    input_size = train_input_seqs[0].shape[1]
    target_size = dataset.num_classes
    
    if gpu_ind is None or 'CUDA_VISIBLE_DEVICES' not in os.environ:
        device = '/cpu:0'
        config = None
    else:
        raw_devices = os.environ['CUDA_VISIBLE_DEVICES'].split(',')
        os.environ['CUDA_VISIBLE_DEVICES'] = raw_devices[gpu_ind]
        device = '/gpu:0'
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=gpu_mem_fraction)
        config = tf.ConfigProto(gpu_options=gpu_options, intra_op_parallelism_threads=2,
                                inter_op_parallelism_threads=2, allow_soft_placement=True)

    with tf.device(device), tf.Graph().as_default(), tf.Session(config=config) as sess:
        model = Model(input_size, target_size, params)
        optimizer = Optimizer(model.loss, params)
        train(sess, model, optimizer, train_input_seqs, train_reset_seqs, train_label_seqs,
              test_input_seqs, test_reset_seqs, test_label_seqs, params)

In [ ]:
params = Params()
run_training(params)